In [1]:
import numpy as np
import tensorflow as tf

import plotly.express as px

In [2]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.5,
    height_shift_range=0.5,
    validation_split=0.2

)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train = train_gen.flow_from_directory(
    '/kaggle/input/10-monkey-species/training/training'
    ,target_size=(224,224)
    ,batch_size=10
    ,seed=123
    ,subset='training'
)


validation_data = train_gen.flow_from_directory(
    '/kaggle/input/10-monkey-species/training/training'
    ,target_size=(224,224)
    ,batch_size=10
    ,seed=123
    ,subset='validation'
)

test = test_gen.flow_from_directory(
    '/kaggle/input/10-monkey-species/validation/validation',
    target_size=(224,224)
    ,batch_size=10
)

Found 881 images belonging to 10 classes.
Found 217 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [3]:
mobilen = tf.keras.applications.MobileNet(
        input_shape=(224,224,3)
        ,pooling='avg'
        ,dropout=0.05
        ,include_top=False)

mobilen.trainable = False

17225924/17225924 [==============================] - 0s 0us/step


In [4]:
inputs = tf.keras.Input(shape = (224,224,3))
x = mobilen(inputs,training = False)
x = tf.keras.layers.Dense(896,activation='relu')(x)
x = tf.keras.layers.Dense(896,activation='relu')(x)
x = tf.keras.layers.Dense(448,activation='relu')(x)
outputs = tf.keras.layers.Dense(10,activation='softmax')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']

)

epochs = 4

with tf.device('GPU:0'):
    hist = model.fit(
        train,
        validation_data=validation_data,
        batch_size=20,
        epochs=epochs,
        callbacks = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss'
            ,patience=3
            ,restore_best_weights=True
            ,verbose = 1
        )

    )

Epoch 1/4
89/89 [==============================] - 54s 576ms/step - loss: 1.5222 - accuracy: 0.4745 - val_loss: 0.8278 - val_accuracy: 0.7051
Epoch 2/4
89/89 [==============================] - 44s 494ms/step - loss: 0.8058 - accuracy: 0.7242 - val_loss: 0.5703 - val_accuracy: 0.7972
Epoch 3/4
89/89 [==============================] - 46s 513ms/step - loss: 0.6319 - accuracy: 0.7809 - val_loss: 0.6772 - val_accuracy: 0.7834
Epoch 4/4
89/89 [==============================] - 45s 500ms/step - loss: 0.5315 - accuracy: 0.8150 - val_loss: 0.6099 - val_accuracy: 0.7788


In [5]:
fig = px.line(
    hist.history,
    y=['loss','val_loss']
)

fig.show()

In [6]:
model.evaluate(test)

28/28 [==============================] - 9s 322ms/step - loss: 0.3168 - accuracy: 0.9154


[0.3167520761489868, 0.9154411554336548]

url : https://www.kaggle.com/datasets/slothkong/10-monkey-species